In [112]:
import re
import codecs

from anki import Collection
from collections import defaultdict

from pinyin import *

In [20]:
from pinyin.chinese_master import *

In [22]:
cedict_path = '/home/eahn/personal/anki-chinese/addon21/user_files/cedict_ts.u8'
cedict = ChineseDict(codecs.open(cedict_path, 'r', 'utf-8'))

In [26]:
col = Collection('/home/eahn/.local/share/Anki2/Eric/collection.anki2')

In [29]:
cids = col.findCards('deck:Mandarin::Heisig card:Reading')

In [105]:
def p_from_defs(entries, loc=0):
    ps = []
    for p, _ in entries:
        if p[loc] not in ps:
            ps.append(p[loc])
    return ps

In [185]:
%%time

pinyin_dict = {}

for cid in cids:
    card = col.getCard(cid)
    note = card.note()
    hanzi = note['Hanzi']
    if 'p' in hanzi or not is_hanzi(hanzi) or note['Keyword'] == '' or note['Keyword'].startswith('p.'):
        continue
    pinyin = note['Pinyin']
    pinyin_dict[hanzi] = {'anki': pinyin, 'note': note}
    
    direct = []
    indirect = defaultdict(int)
    
    success, entries = cedict.lookup(hanzi)
    if success:
        direct = p_from_defs(entries)

    for word, entries in cedict.entries.items():
        if hanzi not in word:
            continue
        for p in p_from_defs(entries.items(), word.index(hanzi)):
            indirect[p] += 1
    
    pinyin_dict[hanzi]['direct'] = direct
    pinyin_dict[hanzi]['indirect'] = indirect

CPU times: user 26.9 s, sys: 4.86 ms, total: 26.9 s
Wall time: 27 s


In [267]:
def from_anki(x):
    rex = re.compile(r'{?(?:<span class=\"tone(\d)\">(\w+)<\/span>,)*<span class=\"tone(\d)\">(\w+)<\/span>}?')
    m = rex.match(x)
    if not m:
        return False, None
    pairs = list(m.groups())
    result = []
    for k in range(len(pairs) // 2):
        tone = pairs[2 * k]
        pinyin = pairs[2 * k + 1]
        if tone is None and pinyin is None:
            continue
        tone = int(tone)
        new_pinyin = pinyin_mton(pinyin)
        assert int(new_pinyin[-1]) % 5 == tone % 5, '{} {} {}'.format(pairs, tone, new_pinyin)
        result.append(new_pinyin)
    return True, result

In [323]:
def to_anki(x):
    around = '{}' if len(x) == 1 else '{{{}}}'
    return around.format(','.join('<span class="tone{}">{}</span>'.format(p[-1], pinyin_ntom(p)) for p in x))

In [309]:
for hanzi, pinyins in pinyin_dict.items():
    direct = pinyins['direct']
    indirect = pinyins['indirect']
    note = pinyins['note']
    if len(direct) + len(indirect) == 0:
        continue
    new_direct = [x for x in direct if x.lower() == x]
    direct = new_direct
    if len(direct) > 0:
        note['Pinyin'] = to_anki(direct)
        note.flush()
    else:
        new_indirect = {}
        for x, count in indirect.items():
            if x == x.lower():
                new_indirect[x] = count
        indirect = new_indirect
        best = sorted(indirect, key=lambda x: indirect[x], reverse=True)
        

明 cedict: ['ming2'] extra ming5 12


In [326]:
indirect

{'ming2': 337, 'ming5': 12}

In [330]:
sorted(indirect, key=lambda x: indirect[x], reverse=True)

['ming2', 'ming5']

In [170]:
for word, entries in cedict.entries.items():
    if hanzi not in word:
        continue
    print(word, word.index(hanzi))
    break